# Observing a Multi-Agent Team

This notebook shows how to observe the messages produced by a multi-agent team in real-time by streaming the output.

## Streaming Team Messages

We will use the team from previous notebooks and stream its messages as it processes a task.

This allows us to monitor the team's progress and understand its behavior.


In [1]:
import asyncio
from rich import print as rprint
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize the model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
)

# Define the agents with distinct roles
researcher = AssistantAgent(
    "researcher",
    model_client=model_client,
    system_message="You are a diligent researcher who gathers relevant information.",
)

analyst = AssistantAgent(
    "analyst",
    model_client=model_client,
    system_message="You analyze information and provide insightful summaries.",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    system_message="You write clear and concise reports based on analysis. Signal completion by including the word 'DONE'.",
)

# Define termination condition
termination_condition = TextMentionTermination("DONE")

# Create the team
team = RoundRobinGroupChat(
    [researcher, analyst, writer],
    termination_condition=termination_condition,
)

In [2]:
from autogen_agentchat.base import TaskResult

async def stream_team_messages():
    async for message in team.run_stream(task="Summarize recent advances in renewable energy technology."):
        if isinstance(message, TaskResult):
            print(f"Team finished with stop reason: {message.stop_reason}")
        else:
            print(f"Message from {message.source}: {message.content}")

await stream_team_messages()

Message from user: Summarize recent advances in renewable energy technology.
Message from researcher: Recent advances in renewable energy technology have focused on improving efficiency, reducing costs, and increasing the scalability and integration of renewable sources into existing energy systems. Here are some key developments:

1. **Solar Power**: 
   - Advances in photovoltaic (PV) technology have led to higher efficiency solar panels, including perovskite solar cells which promise lower production costs and higher efficiency rates.
   - Bifacial solar panels, which capture sunlight on both sides, have become more popular due to their increased energy generation capacity.
   - Floating solar farms have gained traction, particularly in regions with limited land availability, leveraging bodies of water to host the installations.

2. **Wind Power**: 
   - The development of larger, more efficient wind turbines has increased the viability of wind energy, particularly offshore wind far